In [1]:
# Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# Training columns
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [4]:
# Store columns
features = test.columns
labels = train['Transported']

In [5]:
print('Training size:', len(train))
print('Test size:', len(test))

Training size: 8693
Test size: 4277


In [6]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [7]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [8]:
train.isnull().values.sum()

2324

In [9]:
# Check which rows have null values
nan_rows = train[train.isnull().T.any()]

nan_rows

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
10,0008_02,Europa,True,B/1/P,TRAPPIST-1e,34.0,False,0.0,0.0,NaN,0.0,0.0,Altardr Flatic,True
15,0012_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,0.0,0.0,Justie Pooles,False
16,0014_01,Mars,False,F/3/P,55 Cancri e,27.0,False,1286.0,122.0,NaN,0.0,0.0,Flats Eccle,False
23,0020_03,Earth,True,E/0/S,55 Cancri e,29.0,False,0.0,0.0,NaN,0.0,0.0,Mollen Mcfaddennon,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8667,9250_01,Europa,False,E/597/P,TRAPPIST-1e,29.0,False,0.0,2972.0,NaN,28.0,188.0,Chain Reedectied,True
8674,9257_01,NaN,False,F/1892/P,TRAPPIST-1e,13.0,False,39.0,0.0,1085.0,24.0,0.0,Ties Apple,False
8675,9259_01,Earth,NaN,F/1893/P,TRAPPIST-1e,44.0,False,1030.0,1015.0,0.0,11.0,NaN,Annah Gilleyons,True
8684,9274_01,NaN,True,G/1508/P,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,Chelsa Bullisey,True


## PassengerId - no nulls

In [10]:
nan_rows[nan_rows['PassengerId'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


## HomePlanet - 201 rows with null value

Possible to guess by destination?

* TRAPPIST-1e = most common home planet is Earth (52% of passengers with same destination), then split somewhat evenly between Mars and Europa
* 55 Cancri e = most common home planet is Europa (49.2% of passengers with same destination), then Earth, then Mars
* PSO J318.5-22 = most common home planet is Earth (89% of passengers with same destination)

Maybe guess by family member / travelling partner:
* Shared first numbers in passenger_id - should have a match if PassengerID ends with _02+

Otherwise: input a value that indicates missing info?

8/17 Next steps:
* Use this desintation to fill in Earth as home planet: PSO J318.5-22 = most common home planet is Earth (89% of passengers with same destination)
* Look more at the passengers who are in a group - see if this is a common situation and if there is more contextual info to help with assuming home planet is same or not
* Use a placeholder for unknown values if we can't find other contextual clues

Using passenger grouping method:
* 60 passengers have a passenger id group (id ends with _02 - _07)
* Split the original dataset column PassengerID into group number, second id
* Split the original dataset column Name into first name, last name
* If group number, last name, and cabin number are same between passengers in the group and the NaN passenger, assign same home planet
* If all passengers with same group number have same home planet, assign same home planet to the passenger with the NaN
* Verify passengers with ID ending with 01 have no passenger group to extrapolate home planet


In [11]:
no_home = nan_rows[nan_rows['HomePlanet'].isnull()]

In [12]:
train[train['PassengerId'] == '0064_01']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
58,0064_01,Mars,True,F/14/S,TRAPPIST-1e,15.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True


In [13]:
# 141 passengers who are the first passenger in their group
no_home[no_home['PassengerId'].str.endswith('01')]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
113,0119_01,NaN,False,A/0/P,TRAPPIST-1e,39.0,False,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,False
186,0210_01,NaN,True,D/6/P,55 Cancri e,24.0,False,0.0,0.0,NaN,0.0,0.0,Arraid Inicont,True
225,0242_01,NaN,False,F/46/S,TRAPPIST-1e,18.0,False,313.0,1.0,691.0,283.0,0.0,Almone Sté,False
234,0251_01,NaN,True,C/11/S,55 Cancri e,54.0,False,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,True
274,0303_01,NaN,True,G/41/S,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,Oraryn Kirklander,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9084_01,NaN,False,E/582/P,TRAPPIST-1e,25.0,False,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,False
8613,9194_01,NaN,False,E/603/S,55 Cancri e,53.0,False,0.0,4017.0,0.0,13.0,3147.0,NaN,False
8666,9248_01,NaN,False,F/1792/S,55 Cancri e,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,True
8674,9257_01,NaN,False,F/1892/P,TRAPPIST-1e,13.0,False,39.0,0.0,1085.0,24.0,0.0,Ties Apple,False


In [14]:
# 141 passengers who are the first passenger in their group
# Would need to verify that these passengers are not in a larger group, seems unlikely
no_home[no_home['PassengerId'].str.endswith('01')]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
113,0119_01,NaN,False,A/0/P,TRAPPIST-1e,39.0,False,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,False
186,0210_01,NaN,True,D/6/P,55 Cancri e,24.0,False,0.0,0.0,NaN,0.0,0.0,Arraid Inicont,True
225,0242_01,NaN,False,F/46/S,TRAPPIST-1e,18.0,False,313.0,1.0,691.0,283.0,0.0,Almone Sté,False
234,0251_01,NaN,True,C/11/S,55 Cancri e,54.0,False,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,True
274,0303_01,NaN,True,G/41/S,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,Oraryn Kirklander,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9084_01,NaN,False,E/582/P,TRAPPIST-1e,25.0,False,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,False
8613,9194_01,NaN,False,E/603/S,55 Cancri e,53.0,False,0.0,4017.0,0.0,13.0,3147.0,NaN,False
8666,9248_01,NaN,False,F/1792/S,55 Cancri e,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,True
8674,9257_01,NaN,False,F/1892/P,TRAPPIST-1e,13.0,False,39.0,0.0,1085.0,24.0,0.0,Ties Apple,False


In [15]:
train[train['PassengerId'].str.startswith('9084')]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
8515,9084_01,NaN,False,E/582/P,TRAPPIST-1e,25.0,False,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,False


In [16]:
no_home['Destination'].value_counts()

TRAPPIST-1e      150
55 Cancri e       31
PSO J318.5-22     16
Name: Destination, dtype: int64

In [17]:
train['HomePlanet'].value_counts()

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

In [18]:
train['Destination'].value_counts()

TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: Destination, dtype: int64

In [19]:
train['HomePlanet'][train['Destination'] == 'TRAPPIST-1e'].value_counts()

Earth     3101
Mars      1475
Europa    1189
Name: HomePlanet, dtype: int64

In [20]:
train['HomePlanet'][train['Destination'] == '55 Cancri e'].value_counts()

Europa    886
Earth     690
Mars      193
Name: HomePlanet, dtype: int64

In [21]:
train['HomePlanet'][train['Destination'] == 'PSO J318.5-22'].value_counts()

Earth     712
Mars       49
Europa     19
Name: HomePlanet, dtype: int64

## CryoSleep - 217 rows with null value

* Anyone with any value in VIP, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck cannot have been in Cryo sleep - mark these people true
* Cannot necessarily do the same for reverse (87 rows with 0.0 values for all columns) - spending is not high even for awake passengers

8/17 Next steps:
* any spending, CryoSleep == False
* look more at the 0 total spending people to see if they are common or not

Total 0 Spending
* 3247 passengers in training have no spending; majority of non-spenders are asleep
* 470 are awake / 14.47%
* 2690 are asleep / 82.85%
* Groups are inconsistent; not all members of a group necessarily are all awake/asleep

In [ ]:
nan_rows[nan_rows['CryoSleep'].isnull()].sort_values('FoodCourt')

In [ ]:
# People in cryo sleep cannot spend money on ship services
train[train['CryoSleep'] == True].describe()

In [ ]:
# Spending is still low even if you weren't in cryo sleep
train[train['CryoSleep'] == False].describe()

In [ ]:
# There are $0 spenders even if awake
train[train['CryoSleep'] == False]

In [ ]:
# There are 87 rows with no spending and null CryoSleep
nan_rows[nan_rows['CryoSleep'].isnull()][nan_rows['RoomService'] == 0][nan_rows['FoodCourt'] == 0][nan_rows['ShoppingMall'] == 0][nan_rows['Spa'] == 0][nan_rows['VRDeck'] == 0]

In [ ]:
# look at $0 for train, awake
train[train['CryoSleep'] == False][train['RoomService'] == 0][train['FoodCourt'] == 0][train['ShoppingMall'] == 0][train['Spa'] == 0][train['VRDeck'] == 0]

In [ ]:
# look at $0 for train, asleep
train[train['CryoSleep'] == True][train['RoomService'] == 0][train['FoodCourt'] == 0][train['ShoppingMall'] == 0][train['Spa'] == 0][train['VRDeck'] == 0]

In [ ]:
# look at $0 for train overall
train[train['RoomService'] == 0][train['FoodCourt'] == 0][train['ShoppingMall'] == 0][train['Spa'] == 0][train['VRDeck'] == 0]

In [ ]:
train[train['PassengerId'].str.startswith('0031')]

## Cabin - 199 rows with null values

* Doubt there will be any contextual clues other than if they were a paired or grouped passenger, ID ends with _02+
* Check passenger id families and assign same cabin number for at least general location, if cabin location is important for transported outcome

Next steps:
* go by passenger group and age, likely very young kids are sharing cabins with parents, older people would have their own cabin
* would need to look more deeply at different columns to see if there is a relationship or grouping for cabin area - home planet, cryosleep, destination - might lead to cabin grouping

In [ ]:
nan_rows[nan_rows['Cabin'].isnull()]

## Destination - 182 rows with null values

Probably take similar approach to home planet:
* There is correlation between planet and destination
* Grouped passengers may have the same destination
* Add an value representing unknown

8/17 Next steps:
* Could look at cryosleep and see correlation
* May be harder to predict than home planet

CryoSleep relationship:
* More people are awake with TRAPPIST-1e destination (65.9% of passengers going to TRAPPIST)
* Othewise pretty equally split on other destinations, roughly 41% of Cancri destinations are awake, and 48.12% of PSO are awake

TRAPPIST-1e is the most common destination

In [ ]:
nan_rows[nan_rows['Destination'].isnull()]

In [ ]:
train['Destination'][train['CryoSleep'] == True].value_counts()

In [ ]:
train['Destination'][train['CryoSleep'] == False].value_counts()

In [ ]:
train['Destination'].value_counts()

In [ ]:
train[train['PassengerId'].str.startswith('8956')]

## Age - 179 rows with null values

Assign value representing unknown - there are likely no contextual clues that would give us an exact age
* grouped passengers may be similar age, but maybe not

In [ ]:
nan_rows[nan_rows['Age'].isnull()]

## VIP - 203 rows with null values

VIPs are not very common: 2.34% of training dataset
* Might be able to just fill in as False, unless feature is important

VIPs have different spending patterns:
* More likely to be awake (roughly 10% of VIPs are in cryo sleep vs. 35% of regular passengers)
* Very unlikely to have no spending
* Could set a threshold in spending, like 1000 in a category, and then assign VIP or not based on threshold

Could also verify by grouped passengers?

Next step:
* need to look at age of VIPs
* do whole families/groups share VIP status?
* might be ok to assume no VIPs across, though we may want to make exceptions

In [ ]:
nan_rows[nan_rows['VIP'].isnull()]

In [ ]:
train[train['VIP'] == True].describe()

In [ ]:
train[train['VIP'] == False].describe()

In [ ]:
train[train['VIP'] == True][train['CryoSleep'] == False]

In [ ]:
train[train['VIP'] == False]

In [ ]:
train[train['VIP'] == False][train['CryoSleep'] == True]

## Spending - around 200 rows per column, though nulls will be in multiple columns

* If passenger is asleep, fill in 0 for any spending columns (68 rows)
* Could consider filling in 0; spending is not high in general - 50% quintile is like $5 spending
* Otherwise, maybe do an average of spending from other columns? Depends on how important the individual columns are
* Consider combining values from the spending columns into one column if values are correlated with each other and no one column is highly correlated with transported outcome

Next steps:
* assign 0 to people who are asleep


In [ ]:
nan_rows[nan_rows['RoomService'].isnull()]

In [ ]:
nan_rows[nan_rows['RoomService'].isnull()][nan_rows['CryoSleep'] == True]

In [ ]:
train[train['CryoSleep'] == False].describe()

## Name column (200 rows)

* This doesn't seem like it matters; fill in with some value representing unknown
* Unless last name or name contains some kind of context clue on other info, we can't use name for anything
* Would have to do some kind of processing of the name column to see if names indicate something like home planet, VIP, etc. for other missing info

Next steps:
* nothing to do here, probably not predictive of transported
* might want to fill in last names for people with families in case we use last name as an identifier to fill in other missing values?

In [ ]:
nan_rows[nan_rows['Name'].isnull()]

In [ ]:
train[train['PassengerId'].str.startswith('9205')]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
6,0029_01,Europa,True,B/2/P,55 Cancri e,21.0,False,0.0,NaN,0.0,0.0,0.0,Aldah Ainserfle
11,0040_01,Europa,False,D/2/P,TRAPPIST-1e,45.0,False,932.0,74.0,NaN,7.0,1010.0,Antino Pinoffent
15,0046_02,Earth,False,F/11/P,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,903.0,NaN,Glene Powery
18,0047_02,Europa,False,NaN,TRAPPIST-1e,29.0,False,0.0,7708.0,243.0,569.0,343.0,Muons Prucerod
19,0047_03,Europa,False,B/0/S,TRAPPIST-1e,40.0,False,26.0,1925.0,8.0,3144.0,63.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4258,9238_05,Earth,True,NaN,TRAPPIST-1e,14.0,False,0.0,0.0,0.0,0.0,0.0,Caseye Emenez
4267,9260_01,Earth,True,G/1503/P,55 Cancri e,3.0,NaN,0.0,0.0,0.0,0.0,0.0,Luisy Portananney
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore


## Data Clean-up / Filling In Process Ideas

* Split passenger ID into group/number
* Split cabin into deck/num/side
* Split passenger name into first and last
* Add a total spending column that totals the 5 spending columns
